In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('../datasets/variados/movie_small.csv.gz', encoding='utf-8')

In [3]:
df_frases = df['review']

In [4]:
df_label = df['sentiment']

In [5]:
df_frases[3]

'hi for all the people who have seen this wonderful movie im sure thet you would have liked it as much as i. i love the songs once you have seen the show you can sing along as though you are part of the show singing and dancing . dancing and singing. the song ONE is an all time fave musical song too and the strutters at the end with the mirror its so oh you have to watch this one'

In [6]:
df_label.head(n=4)

0    1
1    0
2    0
3    1
Name: sentiment, dtype: int64

In [7]:
num_words = 100

In [8]:
from tensorflow.keras.preprocessing.text import Tokenizer

In [9]:
tokenizer = Tokenizer(num_words=num_words)

In [10]:
df['review'].values[0]

'In 1974, the teenager Martha Moxley (Maggie Grace) moves to the high-class area of Belle Haven, Greenwich, Connecticut. On the Mischief Night, eve of Halloween, she was murdered in the backyard of her house and her murder remained unsolved. Twenty-two years later, the writer Mark Fuhrman (Christopher Meloni), who is a former LA detective that has fallen in disgrace for perjury in O.J. Simpson trial and moved to Idaho, decides to investigate the case with his partner Stephen Weeks (Andrew Mitchell) with the purpose of writing a book. The locals squirm and do not welcome them, but with the support of the retired detective Steve Carroll (Robert Forster) that was in charge of the investigation in the 70\'s, they discover the criminal and a net of power and money to cover the murder.<br /><br />"Murder in Greenwich" is a good TV movie, with the true story of a murder of a fifteen years old girl that was committed by a wealthy teenager whose mother was a Kennedy. The powerful and rich famil

In [11]:
tokenizer.fit_on_texts(df['review'].values)

In [12]:
#tokenizer.word_index

In [16]:
sequences = tokenizer.texts_to_sequences(df['review'].values)

In [14]:
len(sequences[0])

116

In [15]:
len(sequences[1])

125

In [17]:
from tensorflow.keras.preprocessing import sequence

In [18]:
len_seq = 60
seq_pad = sequence.pad_sequences(sequences,maxlen=len_seq)

In [19]:
len(seq_pad[0])

60

In [21]:
seq_pad

array([[ 4,  3,  5, ...,  7,  7, 22],
       [12, 20, 11, ..., 69, 16,  1],
       [37,  1, 66, ...,  7, 43,  4],
       ...,
       [ 2, 17, 84, ..., 10,  9, 32],
       [ 0,  0,  0, ..., 10,  5, 63],
       [ 8, 12,  1, ..., 94, 43,  4]], dtype=int32)

In [20]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Embedding, Dense 

In [20]:
new_dim = 32
model = Sequential()
model.add(Embedding(input_dim=num_words, output_dim=new_dim, input_length=len_seq))
model.add(SimpleRNN(units=10, input_shape=(new_dim,)))
model.add(Dense(1, activation='sigmoid'))

In [21]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 60, 32)            3200      
_________________________________________________________________
simple_rnn (SimpleRNN)       (None, 10)                430       
_________________________________________________________________
dense (Dense)                (None, 1)                 11        
Total params: 3,641
Trainable params: 3,641
Non-trainable params: 0
_________________________________________________________________


In [22]:
size_tr = int(0.8*len(seq_pad))
x_tr, y_tr = seq_pad[:size_tr], df_label.values[:size_tr]
x_test, y_test = seq_pad[size_tr:], df_label.values[size_tr:]

In [23]:
from tensorflow.keras.callbacks import EarlyStopping

In [24]:
callback_early = EarlyStopping(min_delta=0.01, patience=10)

In [25]:
metrics = ['acc']
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=metrics)

In [26]:
history = model.fit(x=x_tr,y=y_tr, batch_size=10, epochs=100,validation_split=0.1, callbacks=[callback_early])

Train on 720 samples, validate on 80 samples
Epoch 1/100
720/720 [==============================] - 7s 10ms/sample - loss: 0.7000 - acc: 0.4764 - val_loss: 0.6875 - val_acc: 0.6000
Epoch 2/100
720/720 [==============================] - 4s 5ms/sample - loss: 0.6643 - acc: 0.6306 - val_loss: 0.6874 - val_acc: 0.5250
Epoch 3/100
720/720 [==============================] - 4s 5ms/sample - loss: 0.6151 - acc: 0.7167 - val_loss: 0.6932 - val_acc: 0.5625
Epoch 4/100
720/720 [==============================] - 4s 5ms/sample - loss: 0.5541 - acc: 0.7653 - val_loss: 0.6717 - val_acc: 0.5375
Epoch 5/100
720/720 [==============================] - 5s 7ms/sample - loss: 0.5067 - acc: 0.7958 - val_loss: 0.6314 - val_acc: 0.6500
Epoch 6/100
720/720 [==============================] - 5s 7ms/sample - loss: 0.4656 - acc: 0.8250 - val_loss: 0.6496 - val_acc: 0.5875
Epoch 7/100
720/720 [==============================] - 6s 8ms/sample - loss: 0.4209 - acc: 0.8500 - val_loss: 0.6907 - val_acc: 0.6125
Epoch 8/1

In [27]:
history.params

{'batch_size': 10,
 'epochs': 100,
 'steps': 72,
 'samples': 720,
 'verbose': 0,
 'do_validation': True,
 'metrics': ['loss', 'acc', 'val_loss', 'val_acc']}

In [28]:
evalu = model.evaluate(x=x_test, y=y_test)
evalu

200/1 [=================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

[0.9256381464004516, 0.585]